In [371]:
from dotenv import load_dotenv
from openai import OpenAI
from elevenlabs.client import AsyncElevenLabs
import asyncio
import json
import os
import argparse
import concurrent.futures

from manga_extraction import extract_all_pages_as_images, save_important_pages, split_volume_into_parts, save_all_pages, extract_panels, scale_base64_image
from vision_analysis import analyze_images_with_gpt4_vision, detect_important_pages, get_important_panels, VISION_PRICE_PER_TOKEN 
from prompts import DRAMATIC_PROMPT, BASIC_PROMPT, BASIC_PROMPT_WITH_CONTEXT,  BASIC_INSTRUCTIONS, KEY_PAGE_IDENTIFICATION_INSTRUCTIONS, KEY_PANEL_IDENTIFICATION_PROMPT, KEY_PANEL_IDENTIFICATION_INSTRUCTIONS
from citation_processing import extract_text_and_citations, extract_script
from movie_director import make_movie
load_dotenv()  # Load environment variables from .env file

True

In [372]:
volume_number = 10
manga = "naruto"

In [373]:
# Initialize OpenAI client with API key
client = OpenAI()
# get elevenlabs api key from dotenv
narration_client = AsyncElevenLabs(api_key=os.getenv("ELEVENLABS_API_KEY"))

print("Extracting all pages from the volume...")
volume_scaled_and_unscaled = extract_all_pages_as_images(f"{manga}/v{volume_number}/v{volume_number}.pdf")
volume = volume_scaled_and_unscaled["scaled"]
volume_unscaled = volume_scaled_and_unscaled["full"]
print("Total pages in volume:", len(volume))

Extracting all pages from the volume...
Total pages in volume: 177


In [374]:
profile_reference = extract_all_pages_as_images(f"{manga}/profile-reference.pdf")["scaled"]
chapter_reference = extract_all_pages_as_images(f"{manga}/chapter-reference.pdf")["scaled"]

profile_pages = []
chapter_pages = [] 

important_page_tokens = 0

batch_size = 20

print("Identifying important pages in the volume...")
# Function to wrap the detect_important_pages call
def process_batch(start_idx, pages):
    response = detect_important_pages(profile_reference, chapter_reference, pages, client,
        KEY_PAGE_IDENTIFICATION_INSTRUCTIONS, KEY_PAGE_IDENTIFICATION_INSTRUCTIONS)
    return start_idx, response

# Using ThreadPoolExecutor to parallelize API calls
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for i in range(0, len(volume), batch_size):
        pages = volume[i:i+batch_size]
        futures.append(executor.submit(process_batch, i, pages))

    for future in concurrent.futures.as_completed(futures):
        start_idx, response = future.result()
        end_index = start_idx + batch_size - 1
        print(f"Processing pages {start_idx} to {min(end_index, len(volume)-1)}")
        
        ip = response["parsed_response"]
        print(json.dumps(ip, indent=2))
        for page in ip:
            if page["type"] == "profile":
                profile_pages.append(page["image_index"] + start_idx)
            elif page["type"] == "chapter":
                chapter_pages.append(page["image_index"] + start_idx)

        important_page_tokens += response["total_tokens"]

profile_pages.sort()
chapter_pages.sort()

print("Total tokens to extract profiles and chapters:", important_page_tokens)
print("\n__________\n")
print("Profile pages:", profile_pages)
print("Chapter pages:", chapter_pages)

Identifying important pages in the volume...
Using GPT as a backup to format JSON object...
Processing pages 160 to 176
[]
Using GPT as a backup to format JSON object...
Processing pages 60 to 79
[
  {
    "image_index": 5,
    "type": "chapter"
  }
]
Using GPT as a backup to format JSON object...
Using GPT as a backup to format JSON object...
Using GPT as a backup to format JSON object...
Processing pages 80 to 99
[
  {
    "image_index": 6,
    "type": "chapter"
  }
]
Using GPT as a backup to format JSON object...
Processing pages 120 to 139
[
  {
    "image_index": 18,
    "type": "chapter"
  }
]
Processing pages 20 to 39
[
  {
    "image_index": 7,
    "type": "chapter"
  }
]
Processing pages 40 to 59
[
  {
    "image_index": 5,
    "type": "chapter"
  }
]
Using GPT as a backup to format JSON object...
Using GPT as a backup to format JSON object...
Processing pages 140 to 159
[
  {
    "image_index": 18,
    "type": "chapter"
  }
]
Using GPT as a backup to format JSON object...
Pro

In [375]:
print(f"{len(volume)}")
print("\n__________\n")
print("Saving important pages to disk for QA...")
save_important_pages(volume, profile_pages, chapter_pages, manga, volume_number)

177

__________

Saving important pages to disk for QA...


In [377]:
character_profiles = [volume[i] for i in profile_pages]    
NUMBER_OF_JOBS = 7
jobs = split_volume_into_parts(volume, volume_unscaled, chapter_pages, NUMBER_OF_JOBS)
parts = jobs["parts"]
jobs_unscaled = jobs["unscaled_images"]
jobs = jobs["scaled_images"]

# Summarize the images in the first job
response = analyze_images_with_gpt4_vision(character_profiles, jobs[0], client, BASIC_PROMPT, BASIC_INSTRUCTIONS)
recap = response.choices[0].message.content
tokens = response.usage.total_tokens
movie_script = extract_text_and_citations(response.choices[0].message.content, jobs[0], jobs_unscaled[0])

print("\n\n\n_____________\n\n\n")
print(response.choices[0].message.content)

# iterate thrugh the rest of the jobs while adding context from previous ones
for i, job in enumerate(jobs):
    if i == 0:
        continue
    response = analyze_images_with_gpt4_vision(character_profiles, job, client, recap + "\n-----\n" + BASIC_PROMPT_WITH_CONTEXT, BASIC_INSTRUCTIONS)
    recap = recap + "\n\n" + response.choices[0].message.content
    tokens += response.usage.total_tokens
    print("\n\n\n_____________\n\n\n")
    print(response.choices[0].message.content)
    movie_script = movie_script + extract_text_and_citations(response.choices[0].message.content, job, jobs_unscaled[i])

print("\n\n\n_____________\n\n\n")
print("\n\n\n_____________\n\n\n")
print("\n\n\n_____________\n\n\n")

narration_script = extract_script(movie_script)
print(narration_script)
print("\n___________\n")

7->27
28->45
46->65
66->86
87->119
120->138
139->end (177)



_____________



In the heat of the Chunin Exam finals, Rock Lee stands firm against the daunting Gaara. The observers underestimate Lee, not knowing "he's stronger than you think" [^2]. As the match commences, Lee's initial attacks, including his "Konoha Hurricane," are effortlessly countered by Gaara's defensive sand [^4]. The sand's relentless defense bewilders Lee's supporters, with even the prodigious Neji noting that "not even his moves are visible" [^7].

Spectators begin to realize Gaara's sand is not merely defensive but "a shield independent of Gaara's will" [^8]. This autonomous defense renders Lee's taijutsu ineffective, leading to astonishment as Gaara boasts "no one has ever wounded me" [^8]. Amidst the fight, a crucial revelation hits—Lee is solely a taijutsu specialist, lacking ninjutsu and genjutsu abilities, a fact met with disbelief by onlookers [^9].

In an unexpected move, Lee's mentor, Might Guy, permit

In [378]:
extract_panels(movie_script)

Load text detector ... Done!
Number of unscaled images in segment: 1
Loading images ... Number of images loaded: 1


Processing images: 1it [00:00, 18.23it/s]


Number of panels generated: 1
Panels generated for text: In the heat of the Chunin Exam finals, Rock Lee stands firm against the daunting Gaara. The observers underestimate Lee, not knowing "he's stronger than you think".
Number of unscaled images in segment: 1
Loading images ... Number of images loaded: 1


Processing images: 1it [00:00, 26.09it/s]


Number of panels generated: 1
Panels generated for text: As the match commences, Lee's initial attacks, including his "Konoha Hurricane," are effortlessly countered by Gaara's defensive sand.
Number of unscaled images in segment: 1
Loading images ... Number of images loaded: 1


...

Number of panels generated: 3
Panels generated for text: The training shifts to a hands-on approach, with Naruto's first task being to walk on water. Despite several humorous mishaps, Naruto's commitment to becoming Hokage drives him to overcome the challenge, showcasing the inspiring determination that defines his character.
Number of unscaled images in segment: 3
Loading images ... Number of images loaded: 3


Processing images: 3it [00:00, 42.40it/s]


Number of panels generated: 3
Panels generated for text: The climax of this training sequence comes when Jiraiya forces Naruto to confront the extraordinary power of the Nine-Tails fox sealed within him. In a dramatic turn, Naruto finds himself grappling with the monster in his psyche. Jiraiya's methods push Naruto to the brink, bringing forth the immense, untamed chakra of the Nine-Tails as a true test of willpower and control.
Number of unscaled images in segment: 3
Loading images ... Number of images loaded: 3


Processing images: 3it [00:00, 39.12it/s]

Number of panels generated: 3
Panels generated for text: As Naruto wrangles with this internal beast, Jiraiya's unorthodox approach reveals a deeper wisdom. Naruto's aspiration to harness the Nine-Tails' power hints at a potential untapped strength that could change the course of his destiny. This intense training arc leaves readers eager for the next chapter, where Naruto's evolution as a ninja promises to unfold further.


In [383]:
print("number of segments:", len(movie_script))
for i, segment in enumerate(movie_script):
    print("segment", i, ": ", segment["text"])
    all_panels_base64 = [panel for sublist in segment["panels"].values() for panel in sublist]
    print(len(all_panels_base64))
    print("number of panels:", len(all_panels_base64))
    print("number of images:", len(segment["images"]))

def process_segment(segment_tuple):
    i, segment = segment_tuple  # Unpack the tuple
    panels = []
    for j, page in enumerate(segment["images"]):
        if "panels" in segment:
            if j not in segment["panels"]:
                panels.append(page)
            else:
                for panel in segment["panels"][j]:
                    panels.append(panel)
        else:
            panels.append(page)
    
    scaled_panels = [scale_base64_image(p) for p in panels]


    response = get_important_panels(profile_reference, scaled_panels, client, 
        segment["text"] + "\n________\n" + KEY_PANEL_IDENTIFICATION_PROMPT, KEY_PANEL_IDENTIFICATION_INSTRUCTIONS)

    important_panels = response["parsed_response"]
    # check if important panels is an array
    if not isinstance(important_panels, list):
        important_panels = []

    ip = []
    for p in important_panels:
        number = p
        if isinstance(number, str):
            if number.isdigit():
                number = int(number)
        if not isinstance(number, int):
            continue

        if number < len(panels):
            ip.append(panels[number])
        
    
    return i, ip, response["total_tokens"]

# Initialize variables
panel_tokens = 0
important_panels_info = {}

# Use ThreadPoolExecutor to parallelize the processing
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Create a list of futures
    futures = [executor.submit(process_segment, (i, segment)) for i, segment in enumerate(movie_script)]
    
    # Collect the results as they complete
    for future in concurrent.futures.as_completed(futures):
        i, ip, tokens = future.result()
        if ip:
            print("Important panels for segment", i, "exist.")
        else: 
            print("No important panels for segment", i)
        movie_script[i]["important_panels"] = ip  # Assign the important panels back to the segment
        panel_tokens += tokens


ELEVENLABS_PRICE_PER_CHARACTER = 0.0003
print("Tokens for extracting profiles and chapters:", important_page_tokens, " | ", "${:,.4f}".format(VISION_PRICE_PER_TOKEN * important_page_tokens))
print("Tokens for summarization:", tokens,  " | ", "${:,.4f}".format(VISION_PRICE_PER_TOKEN * tokens))
print("Tokens for extracting important panels:", panel_tokens, " | ", "${:,.4f}".format(VISION_PRICE_PER_TOKEN * panel_tokens))
total_gpt_tokens = important_page_tokens + tokens + panel_tokens
print("Total GPT tokens:", total_gpt_tokens,  " | ", "${:,.4f}".format(VISION_PRICE_PER_TOKEN * (total_gpt_tokens)))
print("Total elevenlabs characters:", len(narration_script), " | ", "${:,.4f}".format(ELEVENLABS_PRICE_PER_CHARACTER * (len(narration_script))))
print("GRAND TOTAL COST"," | ", "${:,.4f}".format(VISION_PRICE_PER_TOKEN * (total_gpt_tokens) + ELEVENLABS_PRICE_PER_CHARACTER * (len(narration_script))))

number of segments: 56
segment 0 :  In the heat of the Chunin Exam finals, Rock Lee stands firm against the daunting Gaara. The observers underestimate Lee, not knowing "he's stronger than you think".
9
number of panels: 9
number of images: 1
segment 1 :  As the match commences, Lee's initial attacks, including his "Konoha Hurricane," are effortlessly countered by Gaara's defensive sand.
2
number of panels: 2
number of images: 1
segment 2 :  The sand's relentless defense bewilders Lee's supporters, with even the prodigious Neji noting that "not even his moves are visible".
6
number of panels: 6
number of images: 1
segment 3 :  Spectators begin to realize Gaara's sand is not merely defensive but "a shield independent of Gaara's will".
12
number of panels: 12
number of images: 1
segment 4 :  This autonomous defense renders Lee's taijutsu ineffective, leading to astonishment as Gaara boasts "no one has ever wounded me".
12
number of panels: 12
number of images: 1
segment 5 :  Amidst the f

In [384]:
await make_movie(movie_script, manga, volume_number, narration_client)

Narrating movie script...
got bytes: 89861
got bytes: 115774
got bytes: 135000
got bytes: 130821
got bytes: 164675
got bytes: 171363
got bytes: 146285
got bytes: 195186
got bytes: 137508
got bytes: 305110
got bytes: 135000
got bytes: 181812
got bytes: 389955
got bytes: 53498
got bytes: 81502
got bytes: 188499
got bytes: 79412
got bytes: 286720
got bytes: 344816
got bytes: 186409
got bytes: 558811
got bytes: 546690
got bytes: 334785
got bytes: 191425
got bytes: 194351
got bytes: 103235
got bytes: 210233
got bytes: 75650
got bytes: 199784
got bytes: 312633
got bytes: 186409
got bytes: 202710
got bytes: 131239
got bytes: 270419
got bytes: 143360
got bytes: 228623
got bytes: 189335
got bytes: 280868
got bytes: 230295
got bytes: 196858
got bytes: 203546
got bytes: 193097
got bytes: 169273
got bytes: 259970
got bytes: 198948
got bytes: 184737
got bytes: 131239
got bytes: 175124
got bytes: 277942
got bytes: 188499
got bytes: 184737
got bytes: 259970
got bytes: 361534
got bytes: 426318
got byt

MoviePy - Done.
Moviepy - Writing video naruto/v10/recap.mp4



Moviepy - Done !
Moviepy - video ready naruto/v10/recap.mp4
Movie created successfully!


True